In [2]:
import pandas as pd
import numpy as np

### Merging data

In [3]:
xls1 = pd.read_excel(f"raw-data/anid/BDH_PROYECTOS_1982_2002.xlsx")
xls2 = pd.read_excel(f"raw-data/anid/BDH_PROYECTOS_2003_2022.xlsx")
xls3 = pd.read_excel(f"raw-data/anid/BDH_PROYECTOS_2023_2032.xlsx")

xls_unificado = pd.concat( [xls1, xls2, xls3] )
xls_unificado.to_excel(f"anid/ANID-todos.xlsx")

### Add flag for antartic project

In [38]:
xls_unificado = pd.read_excel(f"anid/ANID-todos.xlsx")[['CODIGO_PROYECTO', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'AGNO_FALLO', 'NOMBRE_PROYECTO', 'AREA_OCDE', 'NOMBRE_RESPONSABLE', 'SEXO', 'INSTITUCION_PRINCIPAL', 'MACROZONA_MINCIENCIA', 'REGION_EJECUCION', 'MONTO_ADJUDICADO', 'MONEDA', 'PALABRAS_CLAVES']]
xls_unificado['MONTO_ADJUDICADO'].fillna( 0 )
xls_unificado['PROGRAMA'].fillna( 'noinf' )
xls_unificado['MONEDA'].fillna( 'noinf' )
xls_unificado['REGION_EJECUCION'].fillna( 'noinf' )
xls_unificado['SEXO'].replace( 'HOMBRE', 'M' )
xls_unificado['SEXO'].replace( 'MUJER', 'F' )

xls_anid_antartic_filter = pd.read_excel(f"raw-data/anid/ANID-filtro.xlsx")[['CODIGO_PROYECTO', 'Efectivos']]
xls_procien_antartic = pd.read_excel(f"raw-data/procien/PROCIEN-ANID.xlsx")[['codigo-procien', 'codigo-anid']]

antartic_anid_codes_list = []
antartic_procien_list = []
antartic_anid_list = []
for code, flag in xls_anid_antartic_filter.to_numpy().tolist():
    code = str( code )
    if flag=='si':
        antartic_anid_list.append( code )
        if code not in antartic_anid_codes_list:
            antartic_anid_codes_list.append( code )

for code_procien, code_anid in xls_procien_antartic.to_numpy().tolist():
    code_anid = str( code_anid )
    antartic_procien_list.append( code_anid )
    if code_anid not in antartic_anid_codes_list:
        antartic_anid_codes_list.append( code_anid )

In [39]:
new_xls = []

for row in xls_unificado.to_numpy().tolist():
    new_row = row.copy()
    codigo = str(new_row[0])

    if new_row[14] == 'EUROS':
        ##valor anio 2022
        new_row[13] = 950*float(new_row[13])

    flag_antartic = 'no'
    if codigo in antartic_anid_codes_list: flag_antartic = 'si'

    flag_procien = 'no'
    if codigo in antartic_procien_list: flag_procien = 'si'

    new_xls.append( new_row + [flag_antartic, flag_procien] )

In [40]:
dt = pd.DataFrame( np.array(new_xls), columns=['CODIGO_PROYECTO', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'AGNO_FALLO', 'NOMBRE_PROYECTO', 'AREA_OCDE', 'NOMBRE_RESPONSABLE', 'SEXO', 
       'INSTITUCION_PRINCIPAL', 'MACROZONA_MINCIENCIA', 'REGION_EJECUCION', 'MONTO_ADJUDICADO', 
       'MONEDA', 'PALABRAS_CLAVES', 'ANTARTICO', 'PROCIEN'])
dt.to_excel(f"anid/ANID-antartic.xlsx")